Para executar os notebooks mencionados, caso ainda não tenha feito, você precisa implantar um modelo que use o `text-embedding-ada-002` como modelo base e definir o nome da implantação no arquivo `.env` como `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`.

In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

Em seguida, vamos carregar o Índice de Embedding em um DataFrame do Pandas. O Índice de Embedding está armazenado em um arquivo JSON chamado `embedding_index_3m.json`. O Índice de Embedding contém os Embeddings para cada uma das transcrições do YouTube até o final de outubro de 2023.

In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

Em seguida, vamos criar uma função chamada `get_videos` que buscará no Índice de Embedding pela query fornecida. A função retornará os 5 vídeos mais similares à query. A função funciona da seguinte maneira:

1. Primeiro, é criada uma cópia do Índice de Embedding.
2. Em seguida, o Embedding da query é calculado utilizando a API de Embedding da OpenAI.
3. Depois, uma nova coluna chamada `similarity` é criada no Índice de Embedding. A coluna `similarity` contém a similaridade do cosseno entre o Embedding da query e o Embedding de cada segmento de vídeo.
4. Em seguida, o Índice de Embedding é filtrado pela coluna `similarity`. O Índice de Embedding é filtrado para incluir apenas vídeos que tenham uma similaridade do cosseno maior ou igual a 0,75.
5. Finalmente, o Índice de Embedding é ordenado pela coluna `similarity` e os 5 vídeos mais similares são retornados.

In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

Essa função é bem simples, ela apenas imprime os resultados da consulta da query. 

In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. **Carregar o Índice de Embedding:** O Índice de Embedding é carregado em um DataFrame do Pandas.

2. **Entrada da Query pelo Usuário:** O usuário é solicitado a inserir uma query.

3. **Buscar no Índice de Embedding:** A função `get_videos` é chamada para buscar no Índice de Embedding a query inserida pelo usuário.

4. **Exibir os Resultados:** A função `display_results` é chamada para exibir os resultados ao usuário.

5. **Repetir o Processo:** O usuário é solicitado a inserir outra query. Esse processo continua até que o usuário insira `exit`.


![](../02-chat-and-search/images/notebook-search.png)

Na imagem referenciada, você seria solicitado a inserir uma query. Assim que a query for inserida, a aplicação retornará uma lista de vídeos relevantes à query, junto com um link que leva diretamente ao ponto do vídeo onde a resposta à pergunta está localizada.

Aqui estão algumas queries que você pode testar:

- O que é Azure Machine Learning?
- Como funcionam as redes neurais convolucionais?
- O que é uma rede neural?
- Posso usar Jupyter Notebooks com o Azure Machine Learning?
- O que é ONNX?

In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)